# Make All Tables for Main Manuscript

Kendra Wyant  
August 22, 2025

In [ ]:
#| message: false
#| warning: false

suppressPackageStartupMessages(library(tidyverse))
suppressPackageStartupMessages(source("https://github.com/jjcurtin/lab_support/blob/main/format_path.R?raw=true"))
suppressPackageStartupMessages(library(tidyposterior))
library(kableExtra)


Attaching package: 'kableExtra'

The following object is masked from 'package:dplyr':

    group_rows


Attaching package: 'janitor'

The following objects are masked from 'package:stats':

    chisq.test, fisher.test

## Data and calculations

Table 1

In [ ]:
disposition <- read_csv(file.path(path_processed, "disposition.csv"), 
                        col_types = "ccDDcccccccccc")

screen <- read_csv(file.path(path_shared, "screen.csv"), 
                   col_types = cols()) |>
  filter(subid %in% subset(disposition, analysis == "yes")$subid) |> 
  mutate(across(dsm5_1:dsm5_11, ~ recode(., "No" = 0, "Yes" = 1))) |>  
  rowwise() |>  
  mutate(dsm5_total = sum(c(dsm5_1, dsm5_2, dsm5_3, dsm5_4, dsm5_5, dsm5_6, dsm5_7, 
                              dsm5_8, dsm5_9, dsm5_10, dsm5_11))) |>  
  ungroup()

lapses <- read_csv(file.path(path_shared, "lapses.csv"), col_types = cols()) |>
  filter(exclude == FALSE)

# Calcs to make df for table 1 (demographics and clinical characteristics)
n_total <- 151

dem_age <- screen |>
  summarise(mean = as.character(round(mean(dem_1, na.rm = TRUE), 1)),
            SD = as.character(round(sd(dem_1, na.rm = TRUE), 1)),
            min = as.character(min(dem_1, na.rm = TRUE)),
            max = as.character(max(dem_1, na.rm = TRUE))) |>
  mutate(var = "Age",
         n = as.numeric(""),
         perc = as.numeric("")) |>
  select(var, n, perc, everything()) 

dem_sex <-  screen |>
  select(var = dem_2) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |> 
  add_row(var = "Sex Assigned at Birth", .before = 1)

dem_race <- screen |>
  select(var = dem_3) |>
  mutate(var = fct_relevel(factor(var,
                         c("American Indian/Alaska Native", "Asian", "Black/African American",
                           "White/Caucasian", "Other/Multiracial")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Race", .before = 1)
  
  
dem_ethnicity <- screen |>
  select(var = dem_4) |>
  mutate(var = case_when(var == "No, I am not of Hispanic, Latino, or Spanish origin" ~ "No",
                         TRUE ~ "Yes"),
         var = fct_relevel(factor(var, c("Yes", "No")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Hispanic, Latino, or Spanish origin", .before = 1)

dem_education <- screen |>
  select(var = dem_5) |>
  mutate(var = fct_relevel(factor(var,
                         c("Less than high school or GED degree", "High school or GED",
                           "Some college", "2-Year degree", "College degree", "Advanced degree")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Education", .before = 1)

dem_employment <- screen |>
  select(var = dem_6, dem_6_1) |>
  mutate(var = case_when(dem_6_1 == "Full-time" ~ "Employed full-time",
                         dem_6_1 == "Part-time" ~ "Employed part-time",
                         TRUE ~ var)) |>
  mutate(var = fct_relevel(factor(var,
                         c("Employed full-time", "Employed part-time", "Full-time student",
                           "Homemaker", "Disabled", "Retired", "Unemployed",
                           "Temporarily laid off, sick leave, or maternity leave",
                           "Other, not otherwise specified")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |>
  add_row(var = "Employment", .before = 1)

dem_income <- screen |>
  summarise(mean = format(round(mean(dem_7, na.rm = TRUE), 0), big.mark = ","),
            SD = format(round(sd(dem_7, na.rm = TRUE), 0), big.mark = ","),
            min =format(round(min(dem_7, na.rm = TRUE), 0), big.mark = ","),
            max = format(round(max(dem_7, na.rm = TRUE), 0), scientific = FALSE, big.mark = ",")) |>
  mutate(var = "Personal Income",
        n = as.numeric(""),
        perc = as.numeric(""),
        mean = str_c("$", as.character(mean)),
        SD = str_c("$", as.character(SD)),
        min = str_c("$", as.character(min)),
        max = as.character(max)) |>
  select(var, n, perc, everything())

dem_marital <- screen |>
  select(var = dem_8) |>
  mutate(var = case_when(var == "Never Married" ~ "Never married",
                         TRUE ~ var)) |>
  mutate(var = fct_relevel(factor(var,
                         c("Never married", "Married", "Divorced", "Separated",
                           "Widowed")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100) |> 
  add_row(var = "Marital Status", .before = 1)

auh <- screen |>
  summarise(mean = mean(auh_1, na.rm = TRUE),
            SD = sd(auh_1, na.rm = TRUE),
            min = min(auh_1, na.rm = TRUE),
            max = max(auh_1, na.rm = TRUE)) |>
  mutate(var = "Age of first drink",
        n = as.numeric(""),
        perc = as.numeric("")) |>
  select(var, n, perc, everything()) |>
  full_join(screen |>
  summarise(mean = mean(auh_2, na.rm = TRUE),
            SD = sd(auh_2, na.rm = TRUE),
            min = min(auh_2, na.rm = TRUE),
            max = max(auh_2, na.rm = TRUE)) |>
  mutate(var = "Age of regular drinking",
        n = as.numeric(""),
        perc = as.numeric("")) |>
  select(var, n, perc, everything()), by = c("var", "n", "perc", "mean", "SD",
                                             "min", "max")) |>
  full_join(screen |>
  summarise(mean = mean(auh_3, na.rm = TRUE),
            SD = sd(auh_3, na.rm = TRUE),
            min = min(auh_3, na.rm = TRUE),
            max = max(auh_3, na.rm = TRUE)) |>
  mutate(var = "Age at which drinking became problematic",
        n = as.numeric(""),
        perc = as.numeric("")) |>
  select(var, n, perc, everything()), by = c("var", "n", "perc", "mean", "SD",
                                             "min", "max")) |>
  full_join(screen |>
  summarise(mean = mean(auh_4, na.rm = TRUE),
            SD = sd(auh_4, na.rm = TRUE),
            min = min(auh_4, na.rm = TRUE),
            max = max(auh_4, na.rm = TRUE)) |>
  mutate(var = "Age of first quit attempt",
        n = as.numeric(""),
        perc = as.numeric("")) |>
  select(var, n, perc, everything()), by = c("var", "n", "perc", "mean", "SD",
                                             "min", "max")) |>
  full_join(screen |>
  # filter out 2 people with 100 and 365 reported quit attempts - will make footnote in table
  filter(auh_5 < 100) |>
  summarise(mean = mean(auh_5, na.rm = TRUE),
            SD = sd(auh_5, na.rm = TRUE),
            min = min(auh_5, na.rm = TRUE),
            max = max(auh_5, na.rm = TRUE)) |>
  mutate(var = "Number of Quit Attempts*",
        n = as.numeric(""),
        perc = as.numeric("")) |>
  select(var, n, perc, everything()), by = c("var", "n", "perc", "mean", "SD",
                                             "min", "max")) |>
  full_join(screen |>
  select(var = auh_6_1) |>
  mutate(var = case_when(var == "Long-Term Residential Treatment (more than 6 months)" ~ "Long-term residential (6+ months)",
                         TRUE ~ var)) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_6_2) |>
  mutate(var = case_when(var == "Short-Term Residential Treatment (less than 6 months)" ~ "Short-term residential (< 6 months)",
                         TRUE ~ var)) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_6_3) |>
  mutate(var = case_when(var == "Outpatient Treatment" ~ "Outpatient",
                         TRUE ~ var)) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_6_4) |>
  mutate(var = case_when(var == "Individual Counseling" ~ "Individual counseling",
                         TRUE ~ var)) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_6_5) |>
  mutate(var = case_when(var == "Group Counseling" ~ "Group counseling",
                         TRUE ~ var)) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_6_6) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_6_7) |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = auh_7) |>
  mutate(var = fct_relevel(factor(var, c("Yes", "No")))) |>
  group_by(var) |>
  summarise(n = n()) |>
  mutate(perc = (n / sum(n)) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_1) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Tobacco products (cigarettes, chewing tobacco, cigars, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_2) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Cannabis (marijuana, pot, grass, hash, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_3) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Cocaine (coke, crack, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_4) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Amphetamine type stimulants (speed, diet pills, ecstasy, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_5) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Inhalants (nitrous, glue, petrol, paint thinner, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_6) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Sedatives or sleeping pills (Valium, Serepax, Rohypnol, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_7) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Hallucinogens (LSD, acid, mushrooms, PCP, Special K, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |>
  full_join(screen |>
  select(var = assist_2_8) |>
  filter(var != "Never" & !is.na(var)) |>
  mutate(var = "Opioids (heroin, morphine, methadone, codeine, etc.)") |>
  group_by(var) |>
  drop_na() |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100), by = c("var", "n", "perc")) |> 
  mutate(across(perc:SD, ~round(., 1))) |> 
  mutate(across(mean:max, ~as.character(.)))

lapses_per_subid <- screen |>
  select(subid) |>
  left_join(lapses |>
  tabyl(subid) |>
  select(-percent), by = "subid") |>
  mutate(n = if_else(is.na(n), 0, n),
         lapse = if_else(n > 0, "yes", "no"))

dem_aud <- screen |>
  summarise(mean = as.character(round(mean(dsm5_total, na.rm = TRUE), 1)),
            SD = as.character(round(sd(dsm5_total, na.rm = TRUE), 1)),
            min = as.character(min(dsm5_total, na.rm = TRUE)),
            max = as.character(max(dsm5_total, na.rm = TRUE))) |>
  mutate(var = "DSM-5 Alcohol Use Disorder Symptom Count",
         n = as.numeric(""),
         perc = as.numeric("")) |>
  select(var, n, perc, everything()) 

lapses_per_subid <- screen |>
  select(subid) |>
  left_join(lapses |>
  janitor::tabyl(subid) |>
  select(-percent), by = "subid") |>
  mutate(n = if_else(is.na(n), 0, n),
         lapse = if_else(n > 0, "yes", "no"))

lapse_info <- lapses_per_subid |>
  group_by(lapse) |>
  rename(var = lapse) |>
  mutate(var = factor(var, levels = c("yes", "no"), labels = c("Yes", "No"))) |>
  summarise(n = n()) |>
  mutate(perc = (n / n_total) * 100,
         mean = NA_character_,
         SD = NA_character_,
         min = NA_character_,
         max = NA_character_) |>
  full_join(lapses_per_subid |>
  summarise(mean = as.character(round(mean(n), 1)),
            SD = as.character(round(sd(n), 1)),
            min = as.character(round(min(n), 1)),
            max = as.character(round(max(n), 1))) |>
  mutate(var = "Number of reported lapses"),
  by = c("var", "mean", "SD", "min", "max")) |> 
  add_row(var = "Reported 1 or More Lapse During Study Period", .before = 1)

table_dem <- dem_age |> 
  bind_rows(dem_sex) |> 
  bind_rows(dem_race) |> 
  bind_rows(dem_ethnicity) |> 
  bind_rows(dem_education) |> 
  bind_rows(dem_employment) |> 
  bind_rows(dem_income) |> 
  bind_rows(dem_marital) |> 
  bind_rows(dem_aud) |> 
  bind_rows(auh) |> 
  bind_rows(lapse_info) |> 
  mutate(range = str_c(min, "-", max),
         perc = round(perc, 1)) |> 
  select(-c(min, max)) |> 
  rename(` ` = var,
         N = n,
         `%` = perc,
         M = mean, 
         Range = range)

Table 2

In [ ]:
ci_baseline <- read_csv(here::here(path_models_lag, "contrast_baseline.csv"), 
                        col_types = cols())

ci_lag <- read_csv(here::here(path_models_lag, "contrast_adjacent.csv"), 
                   col_types = cols())
ci_fairness_0 <- read_csv(here::here(path_models_lag, "pp_dem_contrast_all.csv"), 
                   col_types = cols()) |> 
  filter(lag == 0)

ci_fairness_336 <- read_csv(here::here(path_models_lag, "pp_dem_contrast_all.csv"), 
                   col_types = cols()) |> 
  filter(lag == 336)

table_ci <- ci_baseline |> 
  mutate(ci = str_c("[", round(lower, 3), ", ", round(upper, 3), "]"),
         median = as.character(round(median, 3)),
         probability = as.character(round(probability, 3))) |> 
  select(contrast, median, ci, probability) |> 
  add_row(contrast = "Baseline Contrasts", median = "", ci = "", probability = "") |> 
  mutate(contrast = factor(contrast,
                           levels = c("Baseline Contrasts",
                                      "24 vs. 0",
                                      "72 vs. 0",
                                      "168 vs. 0",
                                      "336 vs. 0"),
                           labels = c("Baseline Contrasts",
                                      "1-day vs. No lag",
                                      "3-day vs. No lag",
                                      "1-week vs. No lag",
                                      "2-week vs. No lag"))) |> 
  arrange(contrast) |> 
  rbind(ci_lag |> 
          mutate(ci = str_c("[", round(lower, 3), ", ", round(upper, 3), "]"),
                  median = as.character(round(median, 3)),
         probability = as.character(round(probability, 3))) |>
          select(contrast, median, ci, probability) |> 
          add_row(contrast = "Adjacent Contrasts", median = "", ci = "", 
                  probability = "") |> 
          mutate(contrast = factor(contrast,
                           levels = c("Adjacent Contrasts",
                                      "72 vs. 24",
                                      "168 vs. 72",
                                      "336 vs. 168"),
                           labels = c("Adjacent Contrasts",
                                      "3-day vs. 1-day",
                                      "1-week vs. 3-day",
                                      "2-week vs. 1-week"))) |> 
          arrange(contrast)) |> 
    rename(Contrast = contrast,
         Median = median,
         `Bayesian CI` = ci,
         Probability = probability)
 
  table_ci_fair <- ci_fairness_0 |> 
          mutate(ci = str_c("[", round(lower, 3), ", ", round(upper, 3), "]"),
                  median = as.character(round(median, 3)),
         probability = as.character(round(probability, 3))) |>
          select(contrast, median, ci, probability) |> 
          add_row(contrast = "Fairness Contrasts (No Lag)", median = "", ci = "", 
                  probability = "") |> 
          mutate(contrast = factor(contrast,
                           levels = c("Fairness Contrasts (No Lag)",
                                      "female vs male",
                                      "not white vs non-hispanic white",
                                      "below poverty vs above poverty"),
                           labels = c("Fairness Contrasts (No Lag)",
                                      "female vs. male",
                                      "non-White and/or Hispanic vs. non-Hispanic White",
                                      "below poverty line vs. above poverty line"))) |> 
          arrange(contrast) |> 
    rbind(ci_fairness_336 |> 
          mutate(ci = str_c("[", round(lower, 3), ", ", round(upper, 3), "]"),
                  median = as.character(round(median, 3)),
         probability = as.character(round(probability, 3))) |>
          select(contrast, median, ci, probability) |> 
          add_row(contrast = "Fairness Contrasts (2-week Lag)", median = "", ci = "", 
                  probability = "") |> 
          mutate(contrast = factor(contrast,
                           levels = c("Fairness Contrasts (2-week Lag)",
                                      "female vs male",
                                      "not white vs non-hispanic white",
                                      "below poverty vs above poverty"),
                           labels = c("Fairness Contrasts (2-week Lag)",
                                      "female vs. male",
                                      "non-White and/or Hispanic vs. non-Hispanic White",
                                      "below poverty line vs. above poverty line"))) |> 
          arrange(contrast)) |> 
  rename(Contrast = contrast,
         Median = median,
         `Bayesian CI` = ci,
         Probability = probability)

### Table 1: Demographic and Lapse Characteristics

In [ ]:
#| label: tbl-demohtml
#| tbl-cap: "Demographic and Clinical Characteristics"

footnote_table_dem_a <- "N = 151"
footnote_table_dem_b <- "*Two participants reported 100 or more quit attempts. We removed these outliers prior to calculating the mean (M), standard deviation (SD), and range."


table_dem |> 
  knitr::kable() |> 
  kable_classic() |> 
  kableExtra::group_rows(start_row = 3, end_row = 4) |> 
  kableExtra::group_rows(start_row = 6, end_row = 10) |> 
  kableExtra::group_rows(start_row = 12, end_row = 13) |> 
  kableExtra::group_rows(start_row = 15, end_row = 20) |> 
  kableExtra::group_rows(start_row = 22, end_row = 30) |> 
  kableExtra::group_rows(start_row = 33, end_row = 37) |> 
  kableExtra::group_rows("Alcohol Use Disorder Milestones", 
                         start_row = 39, end_row = 42, bold = FALSE) |> 
  kableExtra::group_rows("Lifetime History of Treatment (Can choose more than 1)", 
                         start_row = 44, end_row = 50, bold = FALSE) |> 
  kableExtra::group_rows("Received Medication for Alcohol Use Disorder", 
                         start_row = 51, end_row = 52, bold = FALSE) |> 
  kableExtra::group_rows("Current (Past 3 Month) Drug Use", 
                         start_row = 53, end_row = 60, bold = FALSE) |> 
  kableExtra::group_rows(start_row = 62, end_row = 63) |> 
  kableExtra::footnote(general = c(footnote_table_dem_a, footnote_table_dem_b), escape=FALSE)

N 
 % 
 M 
 SD 
 Range 
 
 
 
 
 Age 
 
 
 41 
 11.9 
 21-72 
 
 
 Sex Assigned at Birth 
 
 
 
 
 
 
 
 
 Female 
 74 
 49.0 
 
 
 
 
 
 Male 
 77 
 51.0 
 
 
 
 
 
 Race 
 
 
 
 
 
 
 
 
 American Indian/Alaska Native 
 3 
 2.0 
 
 
 
 
 
 Asian 
 2 
 1.3 
 
 
 
 
 
 Black/African American 
 8 
 5.3 
 
 
 
 
 
 White/Caucasian 
 131 
 86.8 
 
 
 
 
 
 Other/Multiracial 
 7 
 4.6 
 
 
 
 
 
 Hispanic, Latino, or Spanish origin 
 
 
 
 
 
 
 
 
 Yes 
 4 
 2.6 
 
 
 
 
 
 No 
 147 
 97.4 
 
 
 
 
 
 Education 
 
 
 
 
 
 
 
 
 Less than high school or GED degree 
 1 
 0.7 
 
 
 
 
 
 High school or GED 
 14 
 9.3 
 
 
 
 
 
 Some college 
 41 
 27.2 
 
 
 
 
 
 2-Year degree 
 14 
 9.3 
 
 
 
 
 
 College degree 
 58 
 38.4 
 
 
 
 
 
 Advanced degree 
 23 
 15.2 
 
 
 
 
 
 Employment 
 
 
 
 
 
 
 
 
 Employed full-time 
 72 
 47.7 
 
 
 
 
 
 Employed part-time 
 26 
 17.2 
 
 
 
 
 
 Full-time student 
 7 
 4.6 
 
 
 
 
 
 Homemaker 
 1 
 0.7 
 
 
 
 
 
 Disabled 
 7 
 4.6 
 
 
 
 
 
 Retired 
 8 
 5.3 
 
 
 
 
 
 Unemployed 
 18 
 11.9 
 
 
 
 
 
 Temporarily laid off, sick leave, or maternity leave 
 3 
 2.0 
 
 
 
 
 
 Other, not otherwise specified 
 9 
 6.0 
 
 
 
 
 
 Personal Income 
 
 
 $34,298 
 $31,807 
 $0-200,000 
 
 
 Marital Status 
 
 
 
 
 
 
 
 
 Never married 
 67 
 44.4 
 
 
 
 
 
 Married 
 32 
 21.2 
 
 
 
 
 
 Divorced 
 45 
 29.8 
 
 
 
 
 
 Separated 
 5 
 3.3 
 
 
 
 
 
 Widowed 
 2 
 1.3 
 
 
 
 
 
 DSM-5 Alcohol Use Disorder Symptom Count 
 
 
 8.9 
 1.9 
 4-11 
 
 Alcohol Use Disorder Milestones 
 
 Age of first drink 
 
 
 14.6 
 2.9 
 6-24 
 
 
 Age of regular drinking 
 
 
 19.5 
 6.6 
 11-56 
 
 
 Age at which drinking became problematic 
 
 
 27.8 
 9.6 
 15-60 
 
 
 Age of first quit attempt 
 
 
 31.5 
 10.4 
 15-65 
 
 
 Number of Quit Attempts* 
 
 
 5.5 
 5.8 
 0-30 
 
 Lifetime History of Treatment (Can choose more than 1) 
 
 Long-term residential (6+ months) 
 8 
 5.3 
 
 
 
 
 
 Short-term residential (< 6 months) 
 49 
 32.5 
 
 
 
 
 
 Outpatient 
 74 
 49.0 
 
 
 
 
 
 Individual counseling 
 97 
 64.2 
 
 
 
 
 
 Group counseling 
 62 
 41.1 
 
 
 
 
 
 Alcoholics Anonymous/Narcotics Anonymous 
 93 
 61.6 
 
 
 
 
 
 Other 
 40 
 26.5 
 
 
 
 
 Received Medication for Alcohol Use Disorder 
 
 Yes 
 59 
 39.1 
 
 
 
 
 
 No 
 92 
 60.9 
 
 
 
 
 Current (Past 3 Month) Drug Use 
 
 Tobacco products (cigarettes, chewing tobacco, cigars, etc.) 
 84 
 55.6 
 
 
 
 
 
 Cannabis (marijuana, pot, grass, hash, etc.) 
 66 
 43.7 
 
 
 
 
 
 Cocaine (coke, crack, etc.) 
 18 
 11.9 
 
 
 
 
 
 Amphetamine type stimulants (speed, diet pills, ecstasy, etc.) 
 15 
 9.9 
 
 
 
 
 
 Inhalants (nitrous, glue, petrol, paint thinner, etc.) 
 3 
 2.0 
 
 
 
 
 
 Sedatives or sleeping pills (Valium, Serepax, Rohypnol, etc.) 
 22 
 14.6 
 
 
 
 
 
 Hallucinogens (LSD, acid, mushrooms, PCP, Special K, etc.) 
 14 
 9.3 
 
 
 
 
 
 Opioids (heroin, morphine, methadone, codeine, etc.) 
 16 
 10.6 
 
 
 
 
 
 Reported 1 or More Lapse During Study Period 
 
 
 
 
 
 
 
 
 Yes 
 84 
 55.6 
 
 
 
 
 
 No 
 67 
 44.4 
 
 
 
 
 
 Number of reported lapses 
 
 
 6.8 
 12 
 0-75 
 
 
 
 Note: 
 
 N = 151 
 
 *Two participants reported 100 or more quit attempts. We removed these outliers prior to calculating the mean (M), standard deviation (SD), and range.

### Table 2: Model Comparisons

In [ ]:
#| label: tbl-model
#| tbl-cap: "Median difference in auROC, 95% Bayesian credible interval (CI), and posterior probability that that the auROC difference was smaller than 0 for all baseline and adjacent lag contrasts."

footnote_table_model <- "Median auROC differences less than 0 indicate the more lagged model, on average, performed worse than the more immediate model (e.g., 1-day lag - no lag, 3-day lag - 1-day lag). Bayesian CI represents the range of values where there is a 95% probability that the true auROC difference lies within that range. Probability indicates the posterior probability that this difference is smaller than 0 (i.e., the models are performing differently)."

table_ci |> 
  knitr::kable() |> 
  kable_classic() |> 
  kableExtra::column_spec(1, width = "25em") |> 
  kableExtra::group_rows(start_row = 2, end_row = 5) |> 
  kableExtra::group_rows(start_row = 7, end_row = 9) |> 
  kableExtra::row_spec(5, extra_css = "border-bottom: 1px solid") |> 
  kableExtra::add_footnote(label = footnote_table_model,
                           notation = "none",
                           escape = FALSE)

Contrast 
 Median 
 Bayesian CI 
 Probability 
 
 
 
 
 Baseline Contrasts 
 
 
 
 
 
 
 1-day vs. No lag 
 -0.021 
 [-0.025, -0.017] 
 1 
 
 
 3-day vs. No lag 
 -0.03 
 [-0.035, -0.025] 
 1 
 
 
 1-week vs. No lag 
 -0.042 
 [-0.048, -0.037] 
 1 
 
 
 2-week vs. No lag 
 -0.062 
 [-0.07, -0.056] 
 1 
 
 
 Adjacent Contrasts 
 
 
 
 
 
 
 3-day vs. 1-day 
 -0.009 
 [-0.013, -0.005] 
 1 
 
 
 1-week vs. 3-day 
 -0.012 
 [-0.017, -0.008] 
 1 
 
 
 2-week vs. 1-week 
 -0.02 
 [-0.026, -0.015] 
 1 
 
 
 
 
 Median auROC differences less than 0 indicate the more lagged model, on average, performed worse than the more immediate model (e.g., 1-day lag - no lag, 3-day lag - 1-day lag). Bayesian CI represents the range of values where there is a 95% probability that the true auROC difference lies within that range. Probability indicates the posterior probability that this difference is smaller than 0 (i.e., the models are performing differently).

### Table 3: Fairness Comparisons

In [ ]:
#| label: tbl-fairness
#| tbl-cap: "Median difference in auROC, 95% Bayesian credible interval (CI), and posterior probability that that the auROC difference was smaller than 0 for fairness contrasts for the no lag and 2-week lagged models."

footnote_table_fair <- "Median auROC differences less than 0 indicate the model, on average, performed worse for the disadvantaged group (female, non-White and/or Hispanic, income below poverty line) compared to the advantaged group (male, non-Hispanic White, income above poverty line). Bayesian CI represents the range of values where there is a 95% probability that the true auROC difference lies within that range. Probability indicates the posterior probability that this difference is smaller than 0 (i.e., the models are performing differently for fairness subgroups)."

table_ci_fair |> 
  knitr::kable() |> 
  kable_classic() |> 
  kableExtra::column_spec(1, width = "25em") |> 
  kableExtra::group_rows(start_row = 2, end_row = 4) |> 
  kableExtra::group_rows(start_row = 6, end_row = 8) |> 
  kableExtra::row_spec(4, extra_css = "border-bottom: 1px solid")  |> 
  kableExtra::add_footnote(label = footnote_table_fair,
                           notation = "none",
                           escape = FALSE)

Contrast 
 Median 
 Bayesian CI 
 Probability 
 
 
 
 
 Fairness Contrasts (No Lag) 
 
 
 
 
 
 
 female vs. male 
 -0.043 
 [-0.059, -0.028] 
 1 
 
 
 non-White and/or Hispanic vs. non-Hispanic White 
 -0.131 
 [-0.222, -0.057] 
 0.999 
 
 
 below poverty line vs. above poverty line 
 -0.012 
 [-0.033, 0.007] 
 0.848 
 
 
 Fairness Contrasts (2-week Lag) 
 
 
 
 
 
 
 female vs. male 
 -0.098 
 [-0.125, -0.073] 
 1 
 
 
 non-White and/or Hispanic vs. non-Hispanic White 
 -0.13 
 [-0.208, -0.058] 
 0.998 
 
 
 below poverty line vs. above poverty line 
 -0.039 
 [-0.073, -0.008] 
 0.98 
 
 
 
 
 Median auROC differences less than 0 indicate the model, on average, performed worse for the disadvantaged group (female, non-White and/or Hispanic, income below poverty line) compared to the advantaged group (male, non-Hispanic White, income above poverty line). Bayesian CI represents the range of values where there is a 95% probability that the true auROC difference lies within that range. Probability indicates the posterior probability that this difference is smaller than 0 (i.e., the models are performing differently for fairness subgroups).